## Marker Selection

In [2]:
import pathlib
from concurrent.futures import ProcessPoolExecutor, as_completed
from itertools import combinations
import numpy as np
import anndata
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import seaborn as sns
import xarray as xr
import subprocess
from sklearn.metrics import roc_auc_score
from cemba_data.tools.hdf5.anndata import rank_features_groups
import warnings
warnings.filterwarnings('ignore')

## Parameter

In [3]:
mcds_pattern = '/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.*.mcds'
min_cluster_cell_number = 10
adj_p_cutoff = 1e-1
top_n = 1000
max_test_cell_population = 100
chunk_size = 1
delta_rate_cutoff = 0.3
auroc_cutoff = 0.8

In [3]:
# Parameters
cluster_col = "MajorSpatial"
tidy_data_path = "/home/hanliu/project/mouse_rostral_brain/study/ClusterMethylMarker/SpetialCellMeta.ITSpatial.msg"
use_clusters = ["IT-L4+MOs", "IT-L23+MOs", "IT-L5+MOs", "IT-L6+MOs", "IT-L5+ORB", "IT-L23+ORB", "IT-L5+PFC", "IT-L6+PFC", "IT-L23+PFC", "IT-L4+MOp", "IT-L23+MOp", "IT-L5+MOp", "IT-L6+MOp", "IT-L6+AI", "IT-L5+AI", "IT-L23+AI", "IT-L6+ACA", "IT-L23+ACA", "IT-L5+ACA", "IT-L23+SSp", "IT-L5+SSp", "IT-L4+SSp", "IT-L6+SSp", "IT-L4+SSs", "IT-L6+SSs", "IT-L5+SSs", "IT-L23+SSs"]
mcds_pattern = "/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.*.mcds"
min_cluster_cell_number = 10
adj_p_cutoff = 0.005
top_n = 10000
cpu = 4
chunk_size=90
max_test_cell_population = 2000
chunk_size = 50
delta_rate_cutoff = 0.3
auroc_cutoff = 0.8


### Stable Parameter

In [4]:
random_seed = 0

## Cell Tidy Data

In [5]:
cell_tidy_data = pd.read_msgpack(tidy_data_path)

if use_clusters is not None:
    cell_tidy_data = cell_tidy_data[cell_tidy_data[cluster_col].isin(
        use_clusters)]
cell_tidy_data.shape[0]

26877

In [6]:
records = []
for cluster, sub_df in cell_tidy_data.groupby(cluster_col):
    if sub_df.shape[0] < max_test_cell_population:
        records.append(sub_df)
    else:
        records.append(
            sub_df.sample(max_test_cell_population, random_state=random_seed))
cell_tidy_data = pd.concat(records)
cell_tidy_data[cluster_col].value_counts()

IT-L4+SSp     2000
IT-L23+MOs    2000
IT-L23+SSp    1967
IT-L23+MOp    1956
IT-L23+ACA    1813
IT-L5+MOp     1387
IT-L23+ORB    1318
IT-L5+MOs     1210
IT-L23+PFC    1209
IT-L6+SSp     1161
IT-L5+ACA     1128
IT-L4+MOp     1064
IT-L5+SSp      978
IT-L6+MOp      840
IT-L6+ACA      787
IT-L5+PFC      778
IT-L23+SSs     697
IT-L4+SSs      695
IT-L6+PFC      644
IT-L6+SSs      561
IT-L6+MOs      513
IT-L23+AI      419
IT-L5+ORB      413
IT-L5+AI       370
IT-L4+MOs      342
IT-L5+SSs      301
IT-L6+AI       187
Name: MajorSpatial, dtype: int64

In [7]:
cluster_series = cell_tidy_data[cluster_col].astype(str)
cluster_counts = cluster_series.value_counts()

def check_cluster(cluster, count):
    if count < min_cluster_cell_number:
        return False
    return True

unique_clusters = [
    cluster for cluster, count in cluster_counts.items()
    if check_cluster(cluster, count)
]
cluster_pairs = list(combinations(unique_clusters, 2))

print(len(unique_clusters), 'pass filter.')
print(len(cluster_pairs), 'pairwise comparison to test.')

27 pass filter.
351 pairwise comparison to test.


## Gene meta

In [8]:
gene_meta = pd.read_csv(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusterMethylMarker/gencode.vM22.annotation.gene.flat.filtered_white_genes.tsv.gz',
    index_col='gene_id',
    sep='\t')
gene_meta.index.name = 'gene'
gene_name_to_id = {v: k for k, v in gene_meta['gene_name'].iteritems()}
gene_idbase_to_id = {i.split('.')[0]: i for i in gene_meta.index}

## Adata

In [9]:
gene_mcds = xr.open_mfdataset(mcds_pattern)
use_gene = gene_mcds.get_index('gene') & gene_meta.index
gene_meta = gene_meta.reindex(use_gene)

In [10]:
gene_mcds

<xarray.Dataset>
Dimensions:           (cell: 104340, gene: 55487, mc_type: 2)
Coordinates:
  * mc_type           (mc_type) object 'CGN' 'CHN'
    strand_type       <U4 'both'
  * gene              (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000064372.1'
    geneslop2k_chrom  (gene) object 'chr1' 'chr1' 'chr1' ... 'chrM' 'chrM'
    geneslop2k_start  (gene) int64 3071252 3100015 3203900 ... 12144 13288 13355
    geneslop2k_end    (gene) int64 3076321 3104124 3673497 ... 16299 16299 16299
  * cell              (cell) object '1A_M_0' '1A_M_1' ... '8J_M_1292'
Data variables:
    gene_da           (cell, gene, mc_type) float64 dask.array<shape=(104340, 55487, 2), chunksize=(10000, 55487, 2)>

In [11]:
gene_mcds = gene_mcds['gene_da'].sel(mc_type='CHN',
                                     cell=cell_tidy_data.index,
                                     gene=gene_meta.index)
gene_mcds

<xarray.DataArray 'gene_da' (cell: 26738, gene: 50231)>
dask.array<shape=(26738, 50231), dtype=float64, chunksize=(539, 50231)>
Coordinates:
    mc_type           <U3 'CHN'
    strand_type       <U4 'both'
  * gene              (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000064372.1'
    geneslop2k_chrom  (gene) object 'chr1' 'chr1' 'chr1' ... 'chrM' 'chrM'
    geneslop2k_start  (gene) int64 3071252 3100015 3203900 ... 12144 13288 13355
    geneslop2k_end    (gene) int64 3076321 3104124 3673497 ... 16299 16299 16299
  * cell              (cell) object '4A_M_1000' '4A_M_1006' ... '6C_M_998'

In [12]:
gene_df = pd.DataFrame(gene_mcds.values, 
                       index=gene_mcds.get_index('cell'), 
                       columns=gene_mcds.get_index('gene'))

In [16]:
cluster_mean = gene_df.groupby(cell_tidy_data['MajorSpatial']).mean()

In [17]:
cluster_mean.to_msgpack('ClusterMean.genes.msg')

In [15]:
cell_tidy_data.to_msgpack('TEMP.msg')

In [16]:
gene_mcds.to_netcdf('TEMP.nc')

## Pairwise test

In [12]:
def get_sig_features(rank_gene_dict):
    pvals_adj = pd.DataFrame(rank_gene_dict['pvals_adj'])
    names = pd.DataFrame(rank_gene_dict['names'])
    return pvals_adj, names


def get_delta(cluster, gene):
    row = cluster_mean_gene_df.loc[gene].copy()
    cluster_value = row.pop(cluster)
    other_mean = row[0] # only two cluster
    delta = cluster_value - other_mean
    return delta


def calculate_single_pair(data_path, pair):
    print(pair)
    cluster_a, cluster_b = pair
    output_dir = pathlib.Path(f'TEMP/{cluster_a}')
    output_dir.mkdir(exist_ok=True)
    output_path = output_dir / f'{cluster_b}.msg'
    if output_path.exists():
        return
    
    this_tidy_data = pd.read_msgpack('TEMP.msg')
    this_tidy_data = this_tidy_data[this_tidy_data[cluster_col].isin(pair)]
    
    mcds = xr.open_dataarray(data_path).sel(cell=this_tidy_data.index).load()
    adata = anndata.AnnData(X=mcds.values,
                    obs=pd.DataFrame([], mcds.get_index('cell')),
                    var=pd.DataFrame([], mcds.get_index('gene')))
    adata.obs['cluster'] = this_tidy_data[cluster_col].astype('category')
    
    # reverse_adata, centered by 1 because after normalization all prior is center to 1
    adata.X = (adata.X - 1) * -1 + 1
    
    # calculate cluster delta
    records = {}
    for cluster, sub_df in adata.obs.groupby('cluster'):
        sub_adata = adata[sub_df.index, :]
        gene_mean = sub_adata.X.mean(axis=0)
        records[cluster] = pd.Series(gene_mean, index=sub_adata.var_names)
    cluster_mean_gene_df = pd.DataFrame(records)

    # judge gene by delta
    cluster_delta = (cluster_mean_gene_df[cluster_a] -
                     cluster_mean_gene_df[cluster_b]).abs()
    delta_judge = cluster_delta > delta_rate_cutoff
    use_adata = adata[:, delta_judge]
    
    sc.tl.rank_genes_groups(use_adata,
                            groupby='cluster',
                            n_genes=top_n,
                            method='wilcoxon')
    pvals_adj, names = get_sig_features(
        use_adata.uns['rank_genes_groups'])

    results = []
    for col in use_adata.obs['cluster'].unique():
        if col not in pair:
            continue
        df = pd.DataFrame({
            'pvals_adj': pvals_adj[col].tolist(),
            'gene_id': names[col].tolist()
        })
        df['cluster_from'] = col
        df['cluster_to'] = cluster_a if col == cluster_b else cluster_b
        results.append(df)
    
    # get total results filter by adj_p
    total_results = pd.concat(results)
    total_results['gene_name'] = total_results['gene_id'].map(
        gene_meta['gene_name'])
    total_results['-lgp'] = -np.log10(total_results['pvals_adj'])
    total_results['-lgp'] = total_results['-lgp'].replace(np.inf, 1000)
    total_results = total_results[
        total_results['pvals_adj'] < adj_p_cutoff].copy()

    # judge by auroc
    total_results['AUROC'] = total_results[[
        'gene_id', 'cluster_from'
    ]].apply(lambda i: get_auroc(i['gene_id'], i['cluster_from'],
                                 use_adata),
             axis=1)
    total_results = total_results[total_results['AUROC'] > auroc_cutoff]
    total_results.to_msgpack(output_path)
    return


def get_auroc(gene_id, cluster, adata):
    yscore = adata.obs_vector(gene_id)
    ylabel = adata.obs['cluster'] == cluster
    score = roc_auc_score(ylabel, yscore)
    score = abs(score - 0.5) + 0.5
    return score

In [10]:
data_path = 'TEMP.nc'

## Run pairwise marker

In [ ]:
pair_marker_counts = {}
n = 0
for chunk_start in range(0, len(cluster_pairs), chunk_size):
    pairs = cluster_pairs[chunk_start : chunk_start + chunk_size]
    
    with ProcessPoolExecutor(cpu) as executor:
        temp_dir = 'TEMP'
        pathlib.Path(temp_dir).mkdir(exist_ok=True)
        futures = []
        for pair in pairs:
            future = executor.submit(calculate_single_pair, data_path, pair)
            futures.append(future)
    
        for future in as_completed(futures):
            n += 1
            if n % 100 == 0:
                print(n)
            future.result()

('IT-L23+MOs', 'IT-L4+SSp')
('IT-L23+MOs', 'IT-L23+SSp')
('IT-L23+MOs', 'IT-L23+MOp')
('IT-L23+MOs', 'IT-L23+ACA')
('IT-L23+MOs', 'IT-L5+MOp')
('IT-L23+MOs', 'IT-L23+ORB')
('IT-L23+MOs', 'IT-L5+MOs')
('IT-L23+MOs', 'IT-L23+PFC')
('IT-L23+MOs', 'IT-L6+SSp')
('IT-L23+MOs', 'IT-L5+ACA')
('IT-L23+MOs', 'IT-L4+MOp')
('IT-L23+MOs', 'IT-L5+SSp')
('IT-L23+MOs', 'IT-L6+MOp')
('IT-L23+MOs', 'IT-L6+ACA')
('IT-L23+MOs', 'IT-L5+PFC')
('IT-L23+MOs', 'IT-L23+SSs')
('IT-L23+MOs', 'IT-L4+SSs')
('IT-L23+MOs', 'IT-L6+PFC')
('IT-L23+MOs', 'IT-L6+SSs')
('IT-L23+MOs', 'IT-L6+MOs')
('IT-L23+MOs', 'IT-L23+AI')
('IT-L23+MOs', 'IT-L5+ORB')
('IT-L23+MOs', 'IT-L5+AI')
('IT-L23+MOs', 'IT-L4+MOs')
('IT-L23+MOs', 'IT-L5+SSs')
('IT-L23+MOs', 'IT-L6+AI')
('IT-L4+SSp', 'IT-L23+SSp')
('IT-L4+SSp', 'IT-L23+MOp')
('IT-L4+SSp', 'IT-L23+ACA')
('IT-L4+SSp', 'IT-L5+MOp')
('IT-L4+SSp', 'IT-L23+ORB')
('IT-L4+SSp', 'IT-L5+MOs')
('IT-L4+SSp', 'IT-L23+PFC')
('IT-L4+SSp', 'IT-L6+SSp')
('IT-L4+SSp', 'IT-L5+ACA')


## Aggregate DEG

In [15]:
temp_dir = pathlib.Path('TEMP')
deg_list = list(temp_dir.glob('**/*msg'))

df_list = []
for path in deg_list:
    df_list.append(pd.read_msgpack(path))

total_markers = pd.concat(df_list)

In [16]:
assert total_markers.set_index(['gene_id', 'cluster_from', 'cluster_to']).index.duplicated().sum() == 0

## Save results

In [17]:
total_markers.to_msgpack('TotalPairwiseMarker.msg')

In [18]:
marker_counts = total_markers.set_index(['cluster_from', 'cluster_to']).index.value_counts()
marker_counts.index = pd.MultiIndex.from_tuples(marker_counts.index.tolist())
marker_counts = marker_counts.reset_index()
marker_counts.columns = ['ClusterA', 'ClusterB', 'GeneCount']
marker_counts.to_csv('Cluster_pair_marker_counts.csv', index=None)

In [19]:
marker_counts[marker_counts['GeneCount'] < 10]

,ClusterA,ClusterB,GeneCount
672,IT-L6+AI,IT-L6+MOp,9
673,IT-L6+MOp,IT-L6+AI,9
674,IT-L23+PFC,IT-L23+ACA,7
675,IT-L23+ACA,IT-L23+PFC,7
676,IT-L23+SSp,IT-L23+MOp,4
677,IT-L5+AI,IT-L5+ORB,4
678,IT-L23+MOp,IT-L23+SSp,4
679,IT-L5+ORB,IT-L5+AI,4
680,IT-L5+SSs,IT-L5+SSp,3
681,IT-L5+SSp,IT-L5+SSs,3


In [20]:
subprocess.run('rm -rf TEMP*', shell=True)

CompletedProcess(args='rm -rf TEMP*', returncode=0)

In [21]:
with open('TotalGeneID.txt', 'w') as f:
    for g in total_markers['gene_id'].unique():
        f.write(f'{g}\n')

In [22]:
total_markers['gene_id'].unique().size

4219

In [28]:
total_markers['left_major'] = total_markers['cluster_from'].apply(lambda i: i.split('+')[0])
total_markers['right_major'] = total_markers['cluster_to'].apply(lambda i: i.split('+')[0])